In [6]:
import os
import json
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Alignment, Font
from openpyxl.utils import get_column_letter

# === PATHS ===
json_folder = r"C:\Users\NNadi\Downloads\DKM-HybridSN-GRSL\res\Json_Hybridsn"
output_excel = r"C:\Users\NNadi\Downloads\DKM-HybridSN-GRSL\res\Excels_datasetwise_48models_HybridSN.xlsx"

# === VARIANT MAP ===
variant_map = {
    ("posthoc", "all", "KMeans"): "V1",
    ("posthoc", "all", "MiniBatchKMeans"): "V2",
    ("posthoc", "all", "EFDPC"): "V3",
    ("posthoc", "exclude", "KMeans"): "V4",
    ("posthoc", "exclude", "MiniBatchKMeans"): "V5",
    ("posthoc", "exclude", "EFDPC"): "V6",
    ("partial", "all", "KMeans"): "V7",
    ("partial", "all", "MiniBatchKMeans"): "V8",
    ("partial", "all", "EFDPC"): "V9",
    ("partial", "exclude", "KMeans"): "V10",
    ("partial", "exclude", "MiniBatchKMeans"): "V11",
    ("partial", "exclude", "EFDPC"): "V12",
}

# === METRICS & COLORS ===
metrics = ["No. of Parameters", "Inference time", "Accuracy", "F1 Score", "Recall", "Training Time"]
columns = ["k", "Metric", "Baseline"] + [f"V{i}" for i in range(1, 13)]
color_map = {
    4: "DAEEF3",   # light blue
    8: "B7DEE8",   # blue
    16: "FCD5B4",  # orange
    32: "E6B8B7",  # pink
}

# === CREATE EXCEL WORKBOOK ===
wb = Workbook()
wb.remove(wb.active)  # Remove default sheet

# === LOOP OVER ALL JSON FILES IN FOLDER ===
for file_name in os.listdir(json_folder):
    if not file_name.endswith(".json"):
        continue

    json_file = os.path.join(json_folder, file_name)
    sheet_name = os.path.splitext(file_name)[0][:31]  # Excel sheet name limit

    with open(json_file, "r") as f:
        data = json.load(f)

    df = pd.DataFrame(data)
    df["layer_selection"] = df["layer_selection"].replace({
        "all": "all",
        "exclude_first_last": "exclude",
    })

    # Assign variants
    def get_variant(row):
        key = (row["stage"].lower(), row["layer_selection"].lower(), row["method"])
        return variant_map.get(key, None)

    df["Variant"] = df.apply(get_variant, axis=1)
    df = df.dropna(subset=["Variant"])

    # Compute derived metrics
    df["TrainingTime"] = (
        df["warmup_time"].fillna(0)
        + df["cluster_time"].fillna(0)
        + df["fine_tune_time"].fillna(0)
    )
    df["InferenceTime"] = df["measured_inf_time"]
    k_values = sorted(df["k"].unique())

    # === CREATE SHEET FOR THIS JSON ===
    ws = wb.create_sheet(title=sheet_name)

    # Header
    for col_idx, name in enumerate(columns, 1):
        cell = ws.cell(row=1, column=col_idx, value=name)
        cell.font = Font(bold=True, color="FFFFFF")
        cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
        cell.alignment = Alignment(horizontal="center", vertical="center")

    current_row = 2

    # Write data per k value
    for k in k_values:
        subset = df[df["k"] == k].set_index("Variant")
        fill_color = color_map.get(k, "FFFFFF")
        start_row = current_row

        for metric in metrics:
            ws.cell(row=current_row, column=1, value=k if metric == metrics[0] else "")
            ws.cell(row=current_row, column=2, value=metric)
            ws.cell(row=current_row, column=3, value="")  # Baseline empty

            for vi, variant in enumerate([f"V{i}" for i in range(1, 13)], start=4):
                val = ""
                if variant in subset.index:
                    if metric == "No. of Parameters":
                        val = subset.loc[variant, "compressed_params_effective"]
                    elif metric == "Inference time":
                        val = subset.loc[variant, "InferenceTime"]
                    elif metric == "Accuracy":
                        val = subset.loc[variant, "acc"]
                        val=val*100
                    elif metric == "F1 Score":
                        val = subset.loc[variant, "f1"]
                        val=val*100
                    elif metric == "Recall":
                        val = subset.loc[variant, "recall"]
                        val=val*100
                    elif metric == "Training Time":
                        val = subset.loc[variant, "TrainingTime"]

                ws.cell(row=current_row, column=vi, value=round(val, 4) if val != "" else "")

            # Background fill
            for col in range(1, len(columns) + 1):
                ws.cell(row=current_row, column=col).fill = PatternFill(
                    start_color=fill_color, end_color=fill_color, fill_type="solid"
                )
                ws.cell(row=current_row, column=col).alignment = Alignment(horizontal="center", vertical="center")

            current_row += 1

        # Merge k cells vertically
        ws.merge_cells(start_row=start_row, start_column=1, end_row=current_row - 1, end_column=1)
        ws.cell(row=start_row, column=1).alignment = Alignment(horizontal="center", vertical="center")

    # Auto column width
    for col in range(1, len(columns) + 1):
        max_len = max(len(str(ws.cell(row=r, column=col).value or "")) for r in range(1, ws.max_row + 1))
        ws.column_dimensions[get_column_letter(col)].width = max_len + 2

# === SAVE FINAL EXCEL ===
wb.save(output_excel)
print(f"✅ Final Excel with all JSON tables saved to '{output_excel}'")


✅ Final Excel with all JSON tables saved to 'C:\Users\NNadi\Downloads\DKM-HybridSN-GRSL\res\Excels_datasetwise_48models_HybridSN.xlsx'


In [1]:
import os
import json
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Alignment, Font
from openpyxl.utils import get_column_letter

# === PATHS ===
json_folder = r"C:\Users\NNadi\Downloads\DKM-HybridSN-GRSL\res\Json_SST"
output_excel = r"C:\Users\NNadi\Downloads\DKM-HybridSN-GRSL\res\Excels_datasetwise_48models_SST.xlsx"

# === VARIANT MAP ===
variant_map = {
    ("posthoc", "all", "KMeans"): "V1",
    ("posthoc", "all", "MiniBatchKMeans"): "V2",
    ("posthoc", "all", "EFDPC"): "V3",
    ("posthoc", "exclude", "KMeans"): "V4",
    ("posthoc", "exclude", "MiniBatchKMeans"): "V5",
    ("posthoc", "exclude", "EFDPC"): "V6",
    ("partial", "all", "KMeans"): "V7",
    ("partial", "all", "MiniBatchKMeans"): "V8",
    ("partial", "all", "EFDPC"): "V9",
    ("partial", "exclude", "KMeans"): "V10",
    ("partial", "exclude", "MiniBatchKMeans"): "V11",
    ("partial", "exclude", "EFDPC"): "V12",
}

# === METRICS & COLORS ===
metrics = ["No. of Parameters", "Inference time", "Accuracy", "F1 Score", "Recall", "Training Time"]
columns = ["k", "Metric", "Baseline"] + [f"V{i}" for i in range(1, 13)]
color_map = {
    4: "DAEEF3",   # light blue
    8: "B7DEE8",   # blue
    16: "FCD5B4",  # orange
    32: "E6B8B7",  # pink
}

# === CREATE EXCEL WORKBOOK ===
wb = Workbook()
wb.remove(wb.active)  # Remove default sheet

# === LOOP OVER ALL JSON FILES IN FOLDER ===
for file_name in os.listdir(json_folder):
    if not file_name.endswith(".json"):
        continue

    json_file = os.path.join(json_folder, file_name)
    sheet_name = os.path.splitext(file_name)[0][:31]  # Excel sheet name limit

    with open(json_file, "r") as f:
        data = json.load(f)

    df = pd.DataFrame(data)
    df["layer_selection"] = df["layer_selection"].replace({
        "all": "all",
        "exclude_first_last": "exclude",
    })

    # Assign variants
    def get_variant(row):
        key = (row["stage"].lower(), row["layer_selection"].lower(), row["method"])
        return variant_map.get(key, None)

    df["Variant"] = df.apply(get_variant, axis=1)
    df = df.dropna(subset=["Variant"])

    # Compute derived metrics
    df["TrainingTime"] = (
        df["warmup_time"].fillna(0)
        + df["cluster_time"].fillna(0)
        + df["fine_tune_time"].fillna(0)
    )
    df["InferenceTime"] = df["measured_inf_time"]
    k_values = sorted(df["k"].unique())

    # === CREATE SHEET FOR THIS JSON ===
    ws = wb.create_sheet(title=sheet_name)

    # Header
    for col_idx, name in enumerate(columns, 1):
        cell = ws.cell(row=1, column=col_idx, value=name)
        cell.font = Font(bold=True, color="FFFFFF")
        cell.fill = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
        cell.alignment = Alignment(horizontal="center", vertical="center")

    current_row = 2

    # Write data per k value
    for k in k_values:
        subset = df[df["k"] == k].set_index("Variant")
        fill_color = color_map.get(k, "FFFFFF")
        start_row = current_row

        for metric in metrics:
            ws.cell(row=current_row, column=1, value=k if metric == metrics[0] else "")
            ws.cell(row=current_row, column=2, value=metric)
            ws.cell(row=current_row, column=3, value="")  # Baseline empty

            for vi, variant in enumerate([f"V{i}" for i in range(1, 13)], start=4):
                val = ""
                if variant in subset.index:
                    if metric == "No. of Parameters":
                        val = subset.loc[variant, "compressed_params_effective"]
                    elif metric == "Inference time":
                        val = subset.loc[variant, "InferenceTime"]
                    elif metric == "Accuracy":
                        val = subset.loc[variant, "acc"]
                        val=val*100
                    elif metric == "F1 Score":
                        val = subset.loc[variant, "f1"]
                        val=val*100
                    elif metric == "Recall":
                        val = subset.loc[variant, "recall"]
                        val=val*100
                    elif metric == "Training Time":
                        val = subset.loc[variant, "TrainingTime"]

                ws.cell(row=current_row, column=vi, value=round(val, 4) if val != "" else "")

            # Background fill
            for col in range(1, len(columns) + 1):
                ws.cell(row=current_row, column=col).fill = PatternFill(
                    start_color=fill_color, end_color=fill_color, fill_type="solid"
                )
                ws.cell(row=current_row, column=col).alignment = Alignment(horizontal="center", vertical="center")

            current_row += 1

        # Merge k cells vertically
        ws.merge_cells(start_row=start_row, start_column=1, end_row=current_row - 1, end_column=1)
        ws.cell(row=start_row, column=1).alignment = Alignment(horizontal="center", vertical="center")

    # Auto column width
    for col in range(1, len(columns) + 1):
        max_len = max(len(str(ws.cell(row=r, column=col).value or "")) for r in range(1, ws.max_row + 1))
        ws.column_dimensions[get_column_letter(col)].width = max_len + 2

# === SAVE FINAL EXCEL ===
wb.save(output_excel)
print(f"✅ Final Excel with all JSON tables saved to '{output_excel}'")


C:\Users\NNadi\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\NNadi\AppData\Local\Temp\ipykernel_21492\2703771382.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  + df["cluster_time"].fillna(0)
C:\Users\NNadi\AppData\Local\Temp\ipykernel_21492\2703771382.py:70: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  + df["cluster_time"].fillna(0)
C:\Users\NNadi\AppData\Local\Temp\ipykernel_21492\2703771

✅ Final Excel with all JSON tables saved to 'C:\Users\NNadi\Downloads\DKM-HybridSN-GRSL\res\Excels_datasetwise_48models_SST.xlsx'
